In [123]:
import pandas as pd
import numpy as np

In [124]:
p111 = pd.read_csv(f'Again submission/A1.csv')

In [125]:
p111.head(5)

,ID,M,S
0,0,['100'],['105']
1,1,['100'],['105']
2,2,"['100', '300']",['399']
3,3,['400'],['496']
4,4,['200'],['236']


In [126]:
import re

In [65]:
class_matrix = []

main_class = [100, 200, 300, 400]
sub_class = [101, 102, 103, 104, 105, 109, 110, 115, 119, 120, 121, 122, 123, 127, 128, 130, 131, 133, 199, 235, 236, 238, 241, 245, 248, 252, 254, 258, 259, 299, 361, 399, 401, 494, 495, 496, 499]

with open('IE506_2024_progchallenge_train.txt', 'r') as file:

    for i,line in enumerate(file):
        line_list = line.split(" ")

        M = line_list[0]
        S = line_list[1]

        # Extract numerical values using regular expressions
        M = re.findall(r'\d+', M)
        S = re.findall(r'\d+', S)

        # Convert the extracted numbers to integers
        M = [int(num) for num in M]
        S  = [int(num) for num in S]

        main_class_vec = [1 if label in M else 0 for label in main_class]
        sub_class_vec = [1 if label in S else 0 for label in sub_class]

        # Concatenate main class and subclass vectors
        class_vector = main_class_vec + sub_class_vec
        
        # Append the class vector to the class matrix
        class_matrix.append(class_vector)

y_matrix = np.array(class_matrix)

In [66]:
y_matrix.shape

(200000, 41)

In [67]:
import numpy as np
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [68]:
# Initialize an empty list to store the feature-value pairs
data = []
row_indices = []
col_indices = []

# Read the data file and populate the data list
with open('IE506_2024_progchallenge_train.txt', 'r') as file:
    for i, line in enumerate(file):
        line_parts = line.split(" ")
        for pair in line_parts[2:]:
            feature_id, feature_value = pair.split(':')
            row_indices.append(i)
            col_indices.append(int(feature_id))
            data.append(float(feature_value))

# Create a sparse matrix from the collected data
X_train = csr_matrix((data, (row_indices, col_indices)), shape=(200000, 47236))

In [69]:
X_train.shape

(200000, 47236)

In [70]:
from sklearn.neighbors import NearestNeighbors

In [71]:
import joblib

# Assuming you have already fitted your model
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X_train)

# Save the fitted model to a file
joblib.dump(nn, 'nearest_neighbors_model.pkl')

['nearest_neighbors_model.pkl']

In [72]:
loaded_model = joblib.load('nearest_neighbors_model.pkl')

In [73]:
from scipy.sparse import dok_matrix
 
# Initialize a dictionary of keys (DOK) matrix
X_test_final = dok_matrix((150000, 47236), dtype=np.float32)
 
with open('IE506_2024_progchallenge_test.txt', 'r') as file:
    for i, line in enumerate(file):
        line_list = line.split(" ")
        for s in line_list:
            id_, val = s.split(':')
            id_ = int(id_)
            val = float(val)
            X_test_final[i, id_] = val
 
# Convert to CSR (Compressed Sparse Row) format for efficient row slicing
X_test = X_test_final.tocsr()

In [75]:
X_test.shape

(150000, 47236)

In [127]:
empty_M = []

# empty_S = []

for i in range(150000):
    if p111['M'][i] == '[]' and p111['S'][i] == '[]':
        empty_M.append(i)

    # if p111['S'][i] == '[]':
    #     empty_S.append(i)

print(len(empty_M))

238


In [128]:
empty_OS = []

# empty_S = []

for i in range(150000):
    if p111['M'][i] != '[]' and p111['S'][i] == '[]':
        empty_OS.append(i)

print(len(empty_OS))

1669


In [104]:
import pickle

# Load the model from the file
with open('xgb_classifier_model2.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

print("Model loaded successfully.")

Model loaded successfully.


In [129]:
# Make predictions
y_pred = loaded_model.predict(X_test)

In [131]:
y_pred.shape

(150000, 41)

In [132]:
# i = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20, 21,   22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40]
cls = [100, 200, 300, 400, 101, 102, 103, 104, 105, 109, 110, 115, 119, 120, 121, 122, 123, 127, 128, 130, 131, 133, 199, 235, 236, 238, 241, 245, 248, 252, 254, 258, 259, 299, 361, 399, 401, 494, 495, 496, 499]

In [133]:
for i in empty_M:
    dist, indices = nn.kneighbors(X_test[i].reshape(1, -1))
    j = indices[0][0]
    temp = y_matrix[j]
    M = []
    S = []
    for k in range(41):
        if 0 <= k <= 3:
            if y_matrix[j,k] == 1:
                M.append(str(cls[k]))

        elif 4 <= k :
            if y_matrix[j,k] == 1:
                S.append(str(cls[k]))

    p111.at[i, 'M'] = str(M)
    p111.at[i, 'S'] = str(S)

In [134]:
empty_M = []

# empty_S = []

for i in range(150000):
    if p111['M'][i] == '[]' and p111['S'][i] == '[]':
        empty_M.append(i)

    # if p111['S'][i] == '[]':
    #     empty_S.append(i)

print(len(empty_M))

0


In [136]:
for i in empty_OS:
    dist, indices = nn.kneighbors(X_test[i].reshape(1, -1))
    j = indices[0][0]
    mcls = eval(p111['M'][i])
    S = []

    for m in mcls:
        if int(m) == 100 :
            for k in range(0,4):
                if y_matrix[j,k] == 1:
                    S.append(str(cls[k]))
        if int(m) == 200:
            for k in range(4,23):
                if y_matrix[j,k] == 1:
                    S.append(str(cls[k]))

        if int(m) == 300:
            for k in range(23,34):
                if y_matrix[j,k] == 1:
                    S.append(str(cls[k]))
        if int(m) == 400:
            for k in range(34, 41):
                if y_matrix[j,k] == 1:
                    S.append(str(cls[k]))

    p111.at[i, 'S'] = str(S)

In [137]:
empty_OS = []

# empty_S = []

for i in range(150000):
    if p111['M'][i] != '[]' and p111['S'][i] == '[]':
        empty_OS.append(i)

print(len(empty_OS))

259


In [50]:
# for i in empty_S:
#     dist, indices = nn.kneighbors(X_test[i].reshape(1, -1))
#     j = indices[0][0]
#     y_pred[i, 4:] = y_matrix[j, 4:]

In [95]:
ID = []
M = []
S = []

for i in range(150000):
    ID.append(i)
    temp_M = []
    temp_S = []
    for j in range(41):
        if 0 <= j <= 3:
            if y_pred[i,j] == 1:
                temp_M.append(str(cls[j]))

        elif 4 <= j :
            if y_pred[i,j] == 1:
                temp_S.append(str(cls[j]))

    M.append(temp_M)
    S.append(temp_S)
    

In [96]:
df = pd.DataFrame({'ID' : ID,
                    'M' : M,
                    'S' : S})

In [97]:
df.head()

,ID,M,S
0,0,[100],[105]
1,1,[100],[105]
2,2,"[100, 300]",[399]
3,3,[400],[496]
4,4,[],[236]


In [138]:
import os

directory = "submission"
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame as a CSV file
filename = os.path.join(directory, f"sub66.csv")
p111.to_csv(filename, index=False)
print('Saved csv file')


Saved csv file


In [90]:
M = [100, 200, 300, 400]

In [58]:
p1 = pd.read_csv(f'submission/sub63.csv')

In [59]:
empty_MM = []

empty_SS = []

for i in range(150000):
    if eval(p1['M'][i]) == []:
        empty_MM.append(i)

    if eval(p1['S'][i]) == []:
        empty_SS.append(i)

In [60]:
len(empty_SS) , len(empty_MM)

(0, 0)